# Object Tracking
Is the process of locating a moving object over a period of time.

### Mean Shift Method
1. Pass initial position of our target object and histogram bad projected image to the **mean shift function**
2. As the object moves the histogram bad projected image also changes
3. **mean shift function** moves the window to new location with max probablity density

**DRAWBACKS**
- The size of the window doesnt alter acc to the size of the object
- if the initial position of the object is not known this method cant be used

### Camshift Method (Continuously Adoptive Meanshift Method)
Here the rectangle size continuously changes accoring to the object

In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture('./Videos/vtest.avi')
ret,frame = cap.read()

In [3]:
# set the initial location of target window
x,y,w,h = 250, 220, 40, 90
track_window = (x,y,w,h)

**Histogram back projection**
Creates an img of the same size but of a single channel as of our imput image, where each pixel corresponds to the probablity of that pixel belonging to our object

In [4]:
# set roi for tracking
roi = frame[y:y+h, x:x+w]

# histogram back projected image
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0.,60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0,180])
roi_hist = cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

In [5]:
# Set up the termination criteria
# either 10 iterations or move by at least 1 pt
term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

In [6]:
while True:
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        
        # find the new hist back projected image
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
        
        
        # apply mean shift to get the new location
        # ret,track_window = cv2.meanShift(dst, track_window, term_criteria)
        
        # apply cam shift method
        ret,track_window = cv2.CamShift(dst, track_window, term_criteria)

        
        # draw it on image
        x,y,w,h = track_window
        finalImg = cv2.rectangle(frame, (x,y), (x+w,y+h), 255, 3)
        
        
        cv2.imshow('frame', frame)        
        cv2.imshow('final image', finalImg)
        cv2.imshow('back projected image', dst)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()            